In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST Dataset 
train_dataset = dsets.MNIST(root='../../dataset/mnist', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='../../dataset/mnist', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2), # mnist는 흑백 28,28,1
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)) # kernal size
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 배치사이즈x한줄로 펴기
        out = self.fc(out) # FFN
        return out

#### nn.Sequential

A sequential container. Modules will be added to it in the order they are passed in the constructor.

#### nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)

Applies a 2D convolution over an input signal composed of several input planes.

In the simplest case, the output value of the layer with input size $(N,C_{in},H,W)$ and output $(N,C_{out},H_{out},W_{out})$ can be precisely described as:

$out(N_i,C_{out_j})=bias(C_{out_j})+∑^{C_in−1}_{k=0} weight_{(C_{out_j},k)}⋆input_{(N_i,k)}$

$Input: (N,C_{in},D_{in},H_{in},W_{in})$<br>
$Output: (N,C_{out},D_{out},H_{out},W_{out})$ where<br> $D_{out}=floor((D_{in}+2∗padding[0]−dilation[0]∗(kernel\_size[0]−1)−1)/stride[0]+1)$ $H_{out}=floor((H_{in}+2∗padding[1]−dilation[1]∗(kernel\_size[1]−1)−1)/stride[1]+1)$
$W_{out}=floor((W_{in}+2∗padding[2]−dilation[2]∗(kernel\_size[2]−1)−1)/stride[2]+1)$

* Output Size : N-F/stride+1
* Padding Size : (F-1)/2

In [5]:
cnn = CNN()
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [6]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Iter [100/600] Loss: 0.1769
Epoch [1/5], Iter [200/600] Loss: 0.0908
Epoch [1/5], Iter [300/600] Loss: 0.0773
Epoch [1/5], Iter [400/600] Loss: 0.0765
Epoch [1/5], Iter [500/600] Loss: 0.0367
Epoch [1/5], Iter [600/600] Loss: 0.0780
Epoch [2/5], Iter [100/600] Loss: 0.0228
Epoch [2/5], Iter [200/600] Loss: 0.0263
Epoch [2/5], Iter [300/600] Loss: 0.1470
Epoch [2/5], Iter [400/600] Loss: 0.0780
Epoch [2/5], Iter [500/600] Loss: 0.1043
Epoch [2/5], Iter [600/600] Loss: 0.0292
Epoch [3/5], Iter [100/600] Loss: 0.0050
Epoch [3/5], Iter [200/600] Loss: 0.0273
Epoch [3/5], Iter [300/600] Loss: 0.0362
Epoch [3/5], Iter [400/600] Loss: 0.0610
Epoch [3/5], Iter [500/600] Loss: 0.0881
Epoch [3/5], Iter [600/600] Loss: 0.1060
Epoch [4/5], Iter [100/600] Loss: 0.0142
Epoch [4/5], Iter [200/600] Loss: 0.0245
Epoch [4/5], Iter [300/600] Loss: 0.0116
Epoch [4/5], Iter [400/600] Loss: 0.0305
Epoch [4/5], Iter [500/600] Loss: 0.0247
Epoch [4/5], Iter [600/600] Loss: 0.0108
Epoch [5/5], Ite

In [7]:
# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Trained Model
#torch.save(cnn.state_dict(), 'cnn.pkl')

Test Accuracy of the model on the 10000 test images: 98 %
